In [1]:
import numpy as np 
import pandas as pd 
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from skimage import io

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50 
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv('train.csv')
valid_df = pd.read_csv('test.csv')

In [3]:
train_df.head()

,img_path,class
0,mdb003.pgm.png,dense
1,mdb004.pgm.png,dense
2,mdb033.pgm.png,dense
3,mdb034.pgm.png,dense
4,mdb035.pgm.png,dense


## Setting up the image augmentation from last Lesson: 
Note that this section of the code has been pre-written for you and does not need to be changed, just run. If you would like to change the ImageDataGenerator parameters, feel free.

In [4]:
## This is the image size that VGG16 takes as input
IMG_SIZE = (224, 224)

In [5]:
train_idg = ImageDataGenerator(rescale=1. / 255.0,
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range= 0.1, 
                              width_shift_range=0.1, 
                              rotation_range=20, 
                              shear_range = 0.1,
                              zoom_range=0.1)
raw_dir = 'data_raw'
train_gen = train_idg.flow_from_dataframe(dataframe=train_df, 
                                         directory=raw_dir, 
                                         x_col = 'img_path',
                                         y_col = 'class',
                                         class_mode = 'binary',
                                         target_size = IMG_SIZE, 
                                         batch_size = 9
                                         )

# Note that the validation data should not be augmented! We only want to do some basic intensity rescaling here
val_idg = ImageDataGenerator(rescale=1. / 255.0
                                 )

val_gen = val_idg.flow_from_dataframe(dataframe=valid_df, 
                                         directory=raw_dir, 
                                         x_col = 'img_path',
                                         y_col = 'class',
                                         class_mode = 'binary',
                                         target_size = IMG_SIZE, 
                                         batch_size = 6) ## We've only been provided with 6 validation images

Found 20 validated image filenames belonging to 2 classes.
Found 6 validated image filenames belonging to 2 classes.


In [6]:
## Pull a single large batch of random validation data for testing after each epoch
testX, testY = val_gen.next()

## Now we'll load in VGG16 with pre-trained ImageNet weights: 

In [7]:
model = VGG16(include_top=True, weights='imagenet')
model.summary()

553467904/553467096 [==============================] - 7s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [8]:
transfer_layer = model.get_layer('block5_pool')
vgg_model = Model(inputs=model.input,
                   outputs=transfer_layer.output)

In [9]:
# check before modify
for layer in vgg_model.layers:
    print(layer.name, layer.trainable)

input_1 False
block1_conv1 True
block1_conv2 True
block1_pool True
block2_conv1 True
block2_conv2 True
block2_pool True
block3_conv1 True
block3_conv2 True
block3_conv3 True
block3_pool True
block4_conv1 True
block4_conv2 True
block4_conv3 True
block4_pool True
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True


In [11]:
## Now, choose which layers of VGG16 we actually want to fine-tune
## Here, we'll freeze all but the last convolutional layer
## Add some code here to freeze all but the last convolutional layer:
##### Your code here ######
for layer in vgg_model.layers[:17]:
    layer.trainable = False

In [12]:
## Check to make sure you froze the right ones: 
for layer in vgg_model.layers:
    print(layer.name, layer.trainable)

input_1 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 True
block5_pool True


## Build a simple sequential model using only the VGG16 architecture
Note the code in the cell below has been pre-written for you, you only need to run it

In [13]:
## Build your model using the mostly-frozen VGG16 architecture: 
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(1, activation='relu'))

In [14]:
## Set our optimizer, loss function, and learning rate (you can change the learning rate here if you'd like)
## but otherwise this cell can be run as is
optimizer = Adam(lr=1e-4)
loss = 'binary_crossentropy'
metrics = ['binary_accuracy']

In [15]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [16]:
## Just run a single epoch to see how it does:
new_model.fit_generator(train_gen, 
                                  validation_data = (testX, testY), 
                                  epochs = 5)

Epoch 1/5
3/3 [==============================] - 17s 6s/step - loss: 3.2953 - binary_accuracy: 0.5500 - val_loss: 7.7125 - val_binary_accuracy: 0.5000
Epoch 2/5
3/3 [==============================] - 15s 5s/step - loss: 2.3473 - binary_accuracy: 0.7500 - val_loss: 5.4212 - val_binary_accuracy: 0.5000
Epoch 3/5
3/3 [==============================] - 15s 5s/step - loss: 3.6614 - binary_accuracy: 0.2500 - val_loss: 0.6583 - val_binary_accuracy: 0.6667
Epoch 4/5
3/3 [==============================] - 15s 5s/step - loss: 2.3173 - binary_accuracy: 0.2500 - val_loss: 0.6719 - val_binary_accuracy: 0.8333
Epoch 5/5
3/3 [==============================] - 15s 5s/step - loss: 1.1266 - binary_accuracy: 0.6000 - val_loss: 3.2530 - val_binary_accuracy: 0.5000


In [17]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25089     
Total params: 14,739,777
Trainable params: 2,384,897
Non-trainable params: 12,354,880
_________________________________________________________________


## Let's try another experiment where we add a few more dense layers:

In [22]:
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a couple of dense (aka. fully-connected) layers.
# This is for combining features that the VGG16 model has
# recognized in the image.

##### Your code here ######
new_model.add(Dense(1024, activation='relu'))
new_model.add(Dense(512, activation='relu'))

# Final output layer:
new_model.add(Dense(1, activation='relu'))

In [23]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [24]:
## Just run a single epoch to see how it does:
new_model.fit_generator(train_gen, 
                                  validation_data = (testX, testY), 
                                  epochs = 5)

Epoch 1/5
3/3 [==============================] - 19s 6s/step - loss: 2.1524 - binary_accuracy: 0.4000 - val_loss: 7.6246 - val_binary_accuracy: 0.0000e+00
Epoch 2/5
3/3 [==============================] - 18s 6s/step - loss: 7.6666 - binary_accuracy: 0.0000e+00 - val_loss: 7.6246 - val_binary_accuracy: 0.0000e+00
Epoch 3/5
3/3 [==============================] - 19s 6s/step - loss: 6.1759 - binary_accuracy: 0.0000e+00 - val_loss: 7.6246 - val_binary_accuracy: 0.0000e+00
Epoch 4/5
3/3 [==============================] - 18s 6s/step - loss: 6.1759 - binary_accuracy: 0.0000e+00 - val_loss: 7.6246 - val_binary_accuracy: 0.0000e+00
Epoch 5/5
3/3 [==============================] - 18s 6s/step - loss: 7.6666 - binary_accuracy: 0.0000e+00 - val_loss: 7.6246 - val_binary_accuracy: 0.0000e+00


## Now let's add dropout and another fully connected layer:

In [32]:
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add several fully-connected layers with dropout
##### Your code here ######
new_model.add(Dropout(0.5))
new_model.add(Dense(1024, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(512, activation='relu'))
new_model.add(Dropout(0.5))

# Final output layer
new_model.add(Dense(1, activation='sigmoid'))

In [33]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [34]:
## Just run a single epoch to see how it does:
new_model.fit_generator(train_gen, 
                                  validation_data = (testX, testY), 
                                  epochs = 5)

Epoch 1/5
3/3 [==============================] - 20s 7s/step - loss: 0.7834 - binary_accuracy: 0.4000 - val_loss: 0.6560 - val_binary_accuracy: 0.6667
Epoch 2/5
3/3 [==============================] - 19s 6s/step - loss: 1.2129 - binary_accuracy: 0.3500 - val_loss: 0.6942 - val_binary_accuracy: 0.5000
Epoch 3/5
3/3 [==============================] - 19s 6s/step - loss: 1.1557 - binary_accuracy: 0.4500 - val_loss: 0.8811 - val_binary_accuracy: 0.5000
Epoch 4/5
3/3 [==============================] - 19s 6s/step - loss: 1.5015 - binary_accuracy: 0.4500 - val_loss: 0.6591 - val_binary_accuracy: 0.3333
Epoch 5/5
3/3 [==============================] - 19s 6s/step - loss: 0.5320 - binary_accuracy: 0.7000 - val_loss: 0.7044 - val_binary_accuracy: 0.5000


## Try again

In [29]:
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
new_model.add(Dropout(0.5))

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(1024, activation='relu'))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
new_model.add(Dropout(0.5))

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(512, activation='relu'))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
new_model.add(Dropout(0.5))

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(256, activation='relu'))

# Add a dense (aka. fully-connected) layer.
# Change the activation function to sigmoid 
# so output of the last layer is in the range of [0,1] 
new_model.add(Dense(1, activation='sigmoid'))

In [30]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [31]:
new_model.fit_generator(train_gen, 
                                  validation_data = (testX, testY), 
                                  epochs = 5)

Epoch 1/5
3/3 [==============================] - 20s 7s/step - loss: 0.8258 - binary_accuracy: 0.5000 - val_loss: 0.6835 - val_binary_accuracy: 0.5000
Epoch 2/5
3/3 [==============================] - 19s 6s/step - loss: 0.6872 - binary_accuracy: 0.6000 - val_loss: 0.7407 - val_binary_accuracy: 0.5000
Epoch 3/5
3/3 [==============================] - 19s 6s/step - loss: 0.8648 - binary_accuracy: 0.5500 - val_loss: 0.8966 - val_binary_accuracy: 0.5000
Epoch 4/5
3/3 [==============================] - 19s 6s/step - loss: 0.8387 - binary_accuracy: 0.4000 - val_loss: 0.7478 - val_binary_accuracy: 0.5000
Epoch 5/5
3/3 [==============================] - 18s 6s/step - loss: 0.5644 - binary_accuracy: 0.7500 - val_loss: 0.6587 - val_binary_accuracy: 0.6667


## Fourth try

In [35]:
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add several fully-connected layers with dropout
##### Your code here ######
new_model.add(Dropout(0.5))
new_model.add(Dense(1024, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(512, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(256, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(128, activation='relu'))
new_model.add(Dropout(0.5))

# Final output layer
new_model.add(Dense(1, activation='sigmoid'))

In [36]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [37]:
## Just run a single epoch to see how it does:
new_model.fit_generator(train_gen, 
                                  validation_data = (testX, testY), 
                                  epochs = 5)

Epoch 1/5
3/3 [==============================] - 20s 7s/step - loss: 0.9030 - binary_accuracy: 0.6500 - val_loss: 0.6823 - val_binary_accuracy: 0.6667
Epoch 2/5
3/3 [==============================] - 19s 6s/step - loss: 1.4039 - binary_accuracy: 0.4500 - val_loss: 0.6849 - val_binary_accuracy: 0.8333
Epoch 3/5
3/3 [==============================] - 19s 6s/step - loss: 1.2579 - binary_accuracy: 0.6000 - val_loss: 0.6811 - val_binary_accuracy: 0.6667
Epoch 4/5
3/3 [==============================] - 19s 6s/step - loss: 1.2347 - binary_accuracy: 0.5500 - val_loss: 0.6745 - val_binary_accuracy: 0.8333
Epoch 5/5
3/3 [==============================] - 19s 6s/step - loss: 1.5407 - binary_accuracy: 0.4500 - val_loss: 0.6638 - val_binary_accuracy: 0.5000
